In [50]:
import torch
import os 

# Importing all the packages

import torch
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
from torchvision.transforms import RandomResizedCrop
from torchvision.transforms import RandomHorizontalFlip
from torchvision.transforms import RandomRotation
from torchvision.transforms import Normalize
from torchvision.transforms import Resize
from torchvision.transforms import transforms
from torch.nn import CrossEntropyLoss
from torch.nn import Softmax
from torch.nn import ReLU
from torch.nn import Dropout
from torch import optim
from tqdm import tqdm
from matplotlib import pyplot as plt 
import argparse



In [51]:
import sys
sys.argv=['']
del sys


ap = argparse.ArgumentParser()
ap.add_argument("-m", "--model", type=str, default="resnet", choices=["vgg", "resnet","mobilenet"], help="name of the backbone model")
args = vars(ap.parse_args())

In [52]:
# Specifying mean and SD from imagenet dataset statistics
Mean= [0.485, 0.456, 0.406]
STD= [0.229, 0.224, 0.225]

# Specifying training hyperparameters
Image_size=1080
Batch_size= 128
Pred_batch_size= 4
EPOCHS=16
LR=0.0001

# Determining the device type
Device= torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [53]:
testing_path=r"C:\Users\97158\Desktop\Apziva\Project 4-MonReader\images\testing" 

In [54]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import Subset

# Defining the dataloader function
def get_Dataloader(Dataset, Batch_size, shuffle=True):
    dl=DataLoader(Dataset,Batch_size,shuffle=shuffle)
    return dl

In [55]:
testTransform=Compose([
    Resize(Image_size),
    ToTensor(),
    Normalize(mean=Mean,std=STD)
])

#Creating test Dataset
testDataset=ImageFolder(testing_path,testTransform)

# Initializing the test dataset
testLoader= get_Dataloader(testDataset,Batch_size=Batch_size)

In [56]:
# check if the name of the backbone model is VGG
if args["model"] == "vgg":
	# load VGG-11 model
	baseModel = torch.hub.load("pytorch/vision:v0.10.0", "vgg11",weights='VGG11_Weights.DEFAULT', skip_validation=True)
# otherwise, the backbone model we will be using is a ResNet
elif args["model"] == "resnet":
	# load ResNet 18 model
	baseModel = torch.hub.load("pytorch/vision:v0.10.0", "resnet18",weights='ResNet18_Weights.DEFAULT', skip_validation=True)
elif args['model']=='mobilenet':
    basemodel=torch.hub.load('pytorch/vision:v0.10.0','mobilenet_v2',weights='MobileNet_V2_Weights.DEFAULT',skip_validation=True) 	

Using cache found in C:\Users\97158/.cache\torch\hub\pytorch_vision_v0.10.0


In [57]:
from torch.nn import Linear, Module, Sequential, Conv2d


class Classifier(Module):
    def __init__(self, baseModel, numclasses, model):
        super().__init__()
        self.baseModel = baseModel

        if model == 'vgg':
            # Extracting the number of input features from the last layer of VGG
            num_features = baseModel.classifier[6].out_features
            
            # Defining the extra layer
            self.extra_layer1 = Linear(num_features, 512)  # Assuming 512 output features
            self.relu1 = ReLU(inplace=True)
            self.extra_layer2=Linear(512,256)
            self.relu2=ReLU(inplace=True)
            
            # Defining the final fully connected layer for classification
            self.fc = Linear(256, numclasses)  # Using the output features from the extra layer
        elif model=='resnet':
            
            self.extra_layer=Linear(baseModel.fc.out_features,256)
            self.relu=ReLU(inplace=True)

            # Defining the final fully connected layer for classification
            self.fc=Linear(256,numclasses)

        elif model=='mobilenet':

            num_features=baseModel.classifier[1].out_features

            #Defining extra layer
            self.extra_layer1=Linear(num_features,512)
            self.relu1 = ReLU(inplace=True)
            self.extra_layer2=Linear(512,256)
            self.relu2=ReLU(inplace=True)
            
            # Defining the final fully connected layer for classification
            self.fc = Linear(256, numclasses)  # Using the output features from the extra layer


        # Softmax layer for classification
        self.softmax = Softmax(dim=1)

    def forward(self, x):
        features = self.baseModel(x)
        
        # Applying the extra layer and ReLU activation
        if 'vgg' in self.baseModel.__class__.__name__.lower():
            features = self.extra_layer1(features)
            features=self.relu1(features)
            features=self.extra_layer2(features)
            features = self.relu2(features)

        elif 'resnet' in self.baseModel.__class__.__name__.lower():
            #Applying the extra layer and ReLu activation
            features=self.extra_layer(features)
            features=self.relu(features)    

        elif 'mobilenet' in self.baseModel.__class__.__name__.lower():  
            features = self.extra_layer1(features)
            features=self.relu1(features)
            features=self.extra_layer2(features)
            features = self.relu2(features)

        
        # Passing the features through the final fully connected layer
        logits = self.fc(features)
        output=self.softmax(logits)
        return output        
        

In [58]:
# Defining paths to store trained model
VGG_Model_path= os.path.join("model_output","VGG_model.pth")
RESNET_Model_path= os.path.join("model_output","ResNet_model.pth")
MobileNet_Model_path= os.path.join("model_output","MobileNet_model.pth")

In [59]:
#Loading the model state and inititalizing the loss function
# build the custom model
model = Classifier(baseModel=baseModel.to(Device),numclasses=2, model=args['model'])
model = model.to(Device)


if args['model']=='vgg':
    model.load_state_dict(torch.load(VGG_Model_path))
elif args['model']=='resnet':
    model.load_state_dict(torch.load(RESNET_Model_path))
elif args['model']=='mobilenet':
    model.load_state_dict(torch.load(MobileNet_Model_path))       

In [60]:
import numpy as np
from sklearn.metrics import classification_report

Activation_func=Softmax()

#Turning off autograd for testing evaluation
with torch.no_grad():

# setting the model in evaluation mode
    model.eval()

# initializing list to store predictions
    test_pred=[]

# looping over the test set
    for (x,y) in testLoader:
        x=x.to(Device)

# making predictions and adding them to the lisy above
        logit=model(x)
        #pred=Activation_func(logit)
        #test_pred.extend(pred.argmax(axis=1).cpu().numpy())
        test_pred.extend(logit.argmax(axis=1).cpu().numpy())
        
test_targets=np.array(testDataset.targets)
train_pred=np.array(test_pred)

# generating classification report
print(classification_report(test_targets,train_pred,target_names=testDataset.classes))

              precision    recall  f1-score   support

   0_notflip       0.53      0.53      0.53       307
      1_flip       0.51      0.50      0.50       290

    accuracy                           0.52       597
   macro avg       0.52      0.52      0.52       597
weighted avg       0.52      0.52      0.52       597



In [61]:
# loss_func=CrossEntropyLoss()
# loss_func.to(Device)

# # Initializing Test Data loss 
# TestCorrect=0
# TotalTestLoss=0
# soft=Softmax()

In [62]:
# import numpy as np
# from sklearn.metrics import classification_report

# # Putting the model in evaluation mode with gradients turned off
# with torch.no_grad():
#     model.eval() 

#     test_pred=[]

# # Looping over the test test and sending it to device
#     for (image, targets) in tqdm(testLoader):
#         (image,targets)=(image.to(Device),targets.to(Device))

#         # Making the predictions and calculating the validation loss
        
#         logit=model(image)
#         loss=loss_func(logit, targets)
#         TotalTestLoss += loss.item()


#     # Getting predictions and calculating all the correct predictions
#         pred=soft(logit)
#         test_pred.extend(pred.argmax(axis=1).detach().cpu().numpy())
#         TestCorrect += (pred.argmax(dim=-1)== targets).sum().item()


# test_targets=np.array(testDataset.targets)
# test_pred=np.array(test_pred)

# # generating classification report
# print(classification_report(test_targets,test_pred,target_names=testDataset.classes))